In [1]:
import zipfile
import os

def unzip_folder(zip_file, extract_to):
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

zip_file = 'val.zip'  
extract_to = 'val' 

unzip_folder(zip_file, extract_to)

zip_file = 'test.zip'  
extract_to = 'test'  

unzip_folder(zip_file, extract_to)

zip_file = 'train.zip'  
extract_to = 'train_unzipped' 

#unzip_folder(zip_file, extract_to) 
#al 3lea deja facut

In [8]:
import os
import ujson

def read_json_file(filepath):
    try:
        with open(filepath, 'rb') as f: 
            file_content = f.read()
            json_data = ujson.loads(file_content)  
            filtered_data = {'target': json_data.get('target'), 'src_fm': json_data.get('src_fm')}
            return filtered_data
    except Exception as e:
        print(f"Error processing file {filepath}: {e}")
        return None

def read_json_files(directory):
    file_paths = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.json'):
                file_paths.append(os.path.join(root, file))

    results = []
    for path in file_paths:
        results.append(read_json_file(path))

    results = [data for data in results if data is not None]

    return results

val = read_json_files('val')
print("val done")
test = read_json_files('test')
print("test done")
train = read_json_files('train_unzipped')
print("train done")


val done
test done
train done


In [9]:
train_src, train_tests = list(zip(*[(d['src_fm'], d['target']) for d in train]))
val_src, val_tests = list(zip(*[(d['src_fm'], d['target']) for d in val]))
test_src, test_tests = list(zip(*[(d['src_fm'], d['target']) for d in test]))

In [14]:
import pandas as pd

df = pd.DataFrame({'src': train_src, 'tests': train_tests})
df.to_csv("train.csv", index=False)
df = pd.DataFrame({'src': val_src, 'tests': val_tests})
df.to_csv("val.csv", index=False)
df = pd.DataFrame({'src': test_src, 'tests': test_tests})
df.to_csv("test.csv", index=False)


In [1]:
import pandas as pd
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
val = pd.read_csv('val.csv')
train_src, train_tests = list(train['src']), list(train['tests'])
val_src, val_tests = list(val['src']), list(val['tests'])
test_src, test_tests = list(test['src']), list(test['tests'])

In [13]:
from transformers import BartTokenizer, BartForConditionalGeneration
import torch
from torch.utils.data import TensorDataset, DataLoader
from transformers import AdamW, get_linear_schedule_with_warmup

tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')

max_seq_length = 128
train_inputs = [tokenizer.encode(code, max_length=max_seq_length, truncation=True, padding='max_length') for code in train_src]
val_inputs = [tokenizer.encode(code, max_length=max_seq_length, truncation=True, padding='max_length') for code in val_src]
print("truncated")
input_ids_train = torch.tensor(train_inputs)
input_ids_val = torch.tensor(val_inputs)

train_data = TensorDataset(input_ids_train)
train_dataloader = DataLoader(train_data, batch_size=64, shuffle=True)
epochs = 40
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_dataloader) * epochs)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("i am now reaching the training loop")
for epoch in range(epochs):
    print("before train")
    model.train()
    print("in train mode")
    total_loss = 0
    for step, batch in enumerate(train_dataloader):
        inputs = batch[0].to(device)
        print("inside loop")
        optimizer.zero_grad()
        outputs = model(input_ids=inputs, labels=inputs)
        loss = outputs.loss
        total_loss += loss.item()
        print("calculated loss")
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        print("batch done")

    avg_train_loss = total_loss / len(train_dataloader)
    print(f'Epoch {epoch + 1}/{epochs}')
    print(f'Average Training Loss: {avg_train_loss:.4f}')
    
#am un gpu foarte prost si nu prea vrea sa se antreneze cat de cat rapid(o epoca ia aprox. 30 de min)
#probabil ca se putea face mai bine, dar nu stiu eu.



KeyboardInterrupt



In [ ]:
test_inputs = [tokenizer.encode(code, max_length=max_seq_length, truncation=True, padding='max_length') for code in test_src]

input_ids_test = torch.tensor(test_inputs).to(device)

generated_test_cases = []
for input_ids in input_ids_test:
    input_ids = input_ids.unsqueeze(0)
    with torch.no_grad():
        output = model.generate(input_ids=input_ids, max_length=50, num_beams=5)
    generated_test_case = tokenizer.decode(output[0], skip_special_tokens=True)
    generated_test_cases.append(generated_test_case)

for java_code, generated_test_case in zip(test_java_code, generated_test_cases):
    print(f"Java Code: {java_code}")
    print(f"Generated Test Case: {generated_test_case}\n")

In [ ]:
torch.save(model.state_dict(), 'tss.pth')